In [1]:
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt

def compute_macd(data, short_window=12, long_window=26, signal_window=9):
    data['EMA12'] = data['Adj Close'].ewm(span=short_window, adjust=False).mean()
    data['EMA26'] = data['Adj Close'].ewm(span=long_window, adjust=False).mean()
    data['MACD'] = data['EMA12'] - data['EMA26']
    data['Signal Line'] = data['MACD'].ewm(span=signal_window, adjust=False).mean()
    return data

def compute_rsi(data, window=14):
    delta = data['Adj Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    data['RSI'] = 100 - (100 / (1 + rs))
    return data

def compute_sma(data, window=50):
    data[f'SMA_{window}'] = data['Adj Close'].rolling(window=window).mean()
    return data
    
import matplotlib.pyplot as plt

import yfinance as yf
from datetime import datetime

def macd_sma_rsi(data, sma_window=50):
    data['Signal'] = 0
    buy_condition = (data['MACD'] > data['Signal Line']) & (data['RSI'] < 30) & (data['Adj Close'] > data[f'SMA_{sma_window}'])
    sell_condition = (data['MACD'] < data['Signal Line']) & (data['RSI'] > 70) & (data['Adj Close'] < data[f'SMA_{sma_window}'])
    data.loc[buy_condition, 'Signal'] = 1
    data.loc[sell_condition, 'Signal'] = -1
    return data

def simulate_macd_rsi_strategy(data, initial_cash=500):
    cash = initial_cash
    holdings = 0
    portfolio_value = []

    for i in range(len(data)):
        if data['Signal'].iloc[i] == 1 and cash > data['Adj Close'].iloc[i]:
            shares = cash // data['Adj Close'].iloc[i]
            holdings += shares
            cash -= shares * data['Adj Close'].iloc[i]
        elif data['Signal'].iloc[i] == -1 and holdings > 0:
            cash += holdings * data['Adj Close'].iloc[i]
            holdings = 0

        portfolio_value.append(cash + holdings * data['Adj Close'].iloc[i])
    
    return portfolio_value

def plot_portfolio_values_first(results, title):
    plt.figure(figsize=(14, 7))
    for stock, values in results.items():
        plt.plot(values, label=stock)
    plt.title(title)
    plt.xlabel('Time')
    plt.ylabel('Portfolio Value ($)')
    plt.legend()
    plt.show()

plot_portfolio_values_first(results, 'Portfolio Value by Time (MACD + RSI Strategy)')

stock_symbols = ['VITL', 'USAP', 'BLBD', 'ROAD']
stocks = {}

for symbol in stock_symbols:
    stocks[symbol] = yf.download(symbol, start='2020-01-01', end='2022-12-31')

for symbol, data in stocks.items():
    data = compute_macd(data)
    data = compute_rsi(data)
    data = compute_sma(data)
    data = combine_indicators(data)
    stocks[symbol] = data

results = {}
final_values = {}
for symbol, data in stocks.items():
    portfolio_value = simulate_macd_rsi_strategy(data)
    results[symbol] = portfolio_value
    final_values[symbol] = portfolio_value[-1] 

plot_portfolio_values_first(results, 'Portfolio Value by Time (MACD + RSI + SMA Strategy)')

NameError: name 'results' is not defined